In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
from pandas import DataFrame
import numpy as np
import lxml.html

In [ ]:

#가수마다 번호 부여. 다른 가수 번호 찾아서 바꾸기만 하면 됨

title_l = []
number_l = []

for i in range(1,11):
    url_s = 'https://music.naver.com/artist/track.nhn?artistId=188&page=' + str(i)
    #print(url_s)
    
    res_s = requests.get(url_s, headers = {
    'user-agent': ('Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) '
                   'Chrome/41.0.2228.0 Safari/537.36')})
    
    e_s = lxml.html.fromstring(res_s.text)
    
    title = e_s.cssselect('.track')        
    a = re.compile("[0-9]{2,}")
    
    
    if len(title) == 50:
        for i in range(0,50):
            title_l.append(title[i].text_content())
            number_l.append(a.findall(title[i].get('onclick')))
            
    else:
        for i in range(0,19):
            title_l.append(title[i].text_content())
            number_l.append(a.findall(title[i].get('onclick')))
        
        

In [ ]:
#잘 됬는지 확인. 둘이 len 똑같아야 함
print(len(title_l), len(number_l))

In [ ]:
title_l

In [ ]:
#중복 없애주고
#이중 리스트 풀자

#이중 리스트 안에 있는 중복 없애기
result = []
for i in range(0,len(number_l)):
    result.append(list(set(number_l[i])))

result

In [ ]:
#이중 리스트 풀기
unlist = [y for x in result for y in x]
unlist

In [ ]:
#곡 번호 기준으로 가사 크롤링 하기!
################찐 가사!
lyrics_l = []
for i in unlist:
    url2 = 'https://music.naver.com/lyric/index.nhn?trackId=' + str(i)
    #print(url2)
    response = requests.get(url2)
    html = response.text 
    soup = BeautifulSoup(html, 'html.parser')
    lyrics = soup.select('.section_lyrics')
    #print(lyrics)
    
    for n in lyrics:
        try:
            lyrics_l.append(n.text.strip())
            

        except:
            lyrics_l.append("NULL")


In [ ]:
#결과 확인

lyrics_l
#잘 나온다!

In [ ]:
#결과 확인. 앞에서 title과 number와 같은 값이여야 함
len(lyrics_l)

In [ ]:
##################저장
save_df = DataFrame({"number" : unlist, "title" : title_l, "lyrics" : lyrics_l})


In [ ]:
save_df_final = save_df[save_df.lyrics.apply(lambda x : len(x)) > 100]
save_df_final.to_csv("이문세.csv", index = False, encoding='utf-8-sig')